<a href="https://colab.research.google.com/github/gramajoguadalupe/RNN_quality_requirement/blob/main/RNN_with_Skorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!uname -a

['Linux 402eedaa9873 4.19.112+ #1 SMP Thu Jul 23 08:00:38 PDT 2020 x86_64 x86_64 x86_64 GNU/Linux']

In [ ]:
!!python --version

['Python 3.6.9']

In [ ]:
!sudo pip install skorch
!sudo pip install dstoolbox


In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pandas as pd 
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
import os
import tarfile

In [ ]:
from dstoolbox.transformers import Padder2d
from dstoolbox.transformers import TextFeaturizer
from scipy import stats
from sklearn.datasets import load_files
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from skorch import NeuralNetClassifier
F = nn.functional

In [ ]:
np.random.seed(0)

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 27-08-2020CorpusRE.xlsx to 27-08-2020CorpusRE.xlsx
User uploaded file "27-08-2020CorpusRE.xlsx" with length 147767 bytes


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def pad_features(req_tagged, longitud):
    ''' Devuelve las oraciones taggeadas, donde son agregados zeros a la 
    izquierda para equiparar la longitud estipulada por el parametro longitud.
    Si el requirimiento ya posee el tamaño de lo estipulado esta toma los 
    longitud primeras palabras
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(req_tagged), longitud), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(req_tagged):
        features[i, -len(row):] = np.array(row)[:longitud]
    
    return features

In [ ]:
df = pd.read_excel('27-08-2020CorpusRE.xlsx')

In [ ]:
df.head()

,Unnamed: 0,Requirements,Completo,No Completo,Tokenized,Pos_tagged,Tagged_Req
0,0,The system shall have a MDI form that allows f...,0,1,"['The', 'system', 'shall', 'have', 'a', 'MDI',...","[('The', 'DT'), ('system', 'NN'), ('shall', 'M...","DT,NN,MD,VB,DT,NNP,NN,WDT,VBZ,IN,DT,NN,IN,DT,N..."
1,1,The system shall display Events in a vertical ...,1,0,"['The', 'system', 'shall', 'display', 'Events'...","[('The', 'DT'), ('system', 'NN'), ('shall', 'M...","DT,NN,MD,VB,NNS,IN,DT,JJ,NN,IN,NN,."
2,2,The system shall display the Events in a graph...,1,0,"['The', 'system', 'shall', 'display', 'the', '...","[('The', 'DT'), ('system', 'NN'), ('shall', 'M...","DT,NN,MD,VB,DT,NNS,IN,DT,NN,IN,NN,."
3,3,The Disputes System must be accessible by both...,0,1,"['The', 'Disputes', 'System', 'must', 'be', 'a...","[('The', 'DT'), ('Disputes', 'NNP'), ('System'...","DT,NNP,NNP,MD,VB,JJ,IN,DT,JJ,CC,JJ,NNS,."
4,4,The Disputes System must prevent users from ac...,1,0,"['The', 'Disputes', 'System', 'must', 'prevent...","[('The', 'DT'), ('Disputes', 'NNP'), ('System'...","DT,NNP,NNP,MD,VB,NNS,IN,VBG,DT,NN,NNS,WDT,VBP,..."


In [ ]:
train_on_gpu = torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [ ]:
VOCAB_SIZE = 38  # This is on the low end
MAX_LEN = 100  # Texts are pretty long on average, this is on the low end
USE_CUDA = train_on_gpu  # Set this to False if you don't want to use CUDA
NUM_CV_STEPS = 10  # Number of randomized search steps to perform (cross validation)

In [ ]:
dataset = df

In [ ]:
X, y = dataset['Tagged_Req'], dataset['Completo']
#X = np.asarray([x for x in X])  # decode from bytes

In [ ]:
for text, target in zip(X[:3], y):    
    print("Target: {}".format(target))
    print(f"POS tagged Requirement {text}")
    print()

Target: 0
POS tagged Requirement DT,NN,MD,VB,DT,NNP,NN,WDT,VBZ,IN,DT,NN,IN,DT,NN,CC,DT,NNS,NN,.

Target: 1
POS tagged Requirement DT,NN,MD,VB,NNS,IN,DT,JJ,NN,IN,NN,.

Target: 1
POS tagged Requirement DT,NN,MD,VB,DT,NNS,IN,DT,NN,IN,NN,.



In [ ]:
steps = [
    ('to_idx', TextFeaturizer(max_features = VOCAB_SIZE)),
    ('pad', Padder2d(max_len = MAX_LEN, pad_value = 0, dtype = int)),
]

In [ ]:
Pipeline(steps).fit_transform(X[:])


array([[ 2, 10,  9, ...,  0,  0,  0],
       [ 2, 10,  9, ...,  0,  0,  0],
       [ 2, 10,  9, ...,  0,  0,  0],
       ...,
       [ 2, 11, 11, ...,  0,  0,  0],
       [30,  2, 10, ...,  0,  0,  0],
       [30, 10, 27, ...,  0,  0,  0]])

In [ ]:
class RequirementRNN(nn.Module):
    """
    The RNN model that will be used to perform Requirement analysis.
    """

    def __init__(self, rec_layer_type='lstm', embedding_dim=128, num_units=128, num_layers=2, dropout=0.3):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()
        self.embedding_dim = embedding_dim
        self.rec_layer_type = rec_layer_type.lower()
        self.num_units = num_units
        self.num_layers = num_layers
        self.dropout = dropout   
        
        # embedding and LSTM layers
        self.emb = nn.Embedding(VOCAB_SIZE + 1, embedding_dim=self.embedding_dim)
        rec_layer = {'lstm': nn.LSTM, 'gru': nn.GRU}[self.rec_layer_type]
             
        # linear and sigmoid layers
        self.rec = rec_layer(
            self.embedding_dim, self.num_units, num_layers=num_layers, batch_first=True)
      

        self.output = nn.Linear(self.num_units, 2)
        


    def forward(self, X):
        """
        de que manera se recorre la red neuronal
        ivar: X es el req de entrada        
        """
        embeddings = self.emb(X)
        # from the recurrent layer, only take the activities from the last sequence step
        if self.rec_layer_type == 'gru':
            _, rec_out = self.rec(embeddings)
        else:
            _, (rec_out, _) = self.rec(embeddings)
        rec_out = rec_out[-1]  # take output of last RNN layer transforma la matriz en un vector o reducción dimensional 
        drop = F.dropout(rec_out, p=self.dropout)
        # Remember that the final non-linearity should be softmax, so that our predict_proba
        # method outputs actual probabilities!
        preout = self.output(drop)
        out = F.softmax(preout, dim=-1)
        return out    
    

In [ ]:
X_transformado = Pipeline(steps).fit_transform(X)

In [ ]:
X_transformado[323]

array([30, 10, 27, 25, 10, 27, 25,  5, 10,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [ ]:
X_transformado[322]

array([10, 27, 25, 30, 18, 26, 25, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [ ]:
steps.append(
    ('net', NeuralNetClassifier(
        RequirementRNN,
        device=('cuda' if USE_CUDA else 'cpu'),
        max_epochs=30,
        lr=0.01,
        optimizer=torch.optim.Adam,
    ))
)

In [ ]:
modelo_neuronal =  NeuralNetClassifier(
        RequirementRNN,
        device=('cuda' if USE_CUDA else 'cpu'),
        max_epochs=30,
        lr=0.01,
        optimizer=torch.optim.Adam
    )

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.0338       0.5000        0.7431  0.5301
      2        0.7711       0.5000        0.7144  0.5356
      3        0.6884       0.5000        0.6969  0.5252
      4        0.6870       0.5625        0.6903  0.5286
      5        0.6898       0.5156        0.6950  0.5643
      6        0.6920       0.5156        0.6932  0.5599
      7        0.6937       0.5312        0.6927  0.5363
      8        0.6950       0.5312        0.6904  0.5183
      9        0.6941       0.5312        0.6907  0.5277
     10        0.6897       0.5312        0.6849  0.5340
     11        0.6869       0.5312        0.6803  0.5470
     12        0.6839       0.5312        0.6745  0.5479
     13        0.6709       0.5625        0.6691  0.5327
     14        0.6656       0.6094        0.6543  0.5195
     15        0.6645       0.6094        0.6500  0.5470
     16        0.6602       0.6094        0.6510  0.5233
     17        0.6622       0.5938        0.6543  0.5564
     18        0.6589       0.6250        0.6484  0.5356
     19        0.6576       0.6250        0.6455  0.5651
     20        0.6576       0.6094        0.6448  0.5130
     21        0.6555       0.6562        0.6410  0.5393
     22        0.6539       0.7188        0.6339  0.5197
     23        0.6506       0.7344        0.6245  0.5202
     24        0.6480       0.7344        0.6178  0.5704
     25        0.6398       0.7500        0.6163  0.5357
     26        0.6394       0.7656        0.6086  0.5140
     27        0.6322       0.7344        0.5885  0.5476
     28        0.6190       0.7656        0.5755  0.5371
     29        0.6027       0.7500        0.5544  0.5263
     30        0.5856       0.7656        0.5429  0.5197
CPU times: user 15.9 s, sys: 202 ms, total: 16.1 s
Wall time: 16.3 s
Pipeline(memory=None,
         steps=[('to_idx',
                 TextFeaturizer(analyzer='word', binary=False,
                                decode_error='strict',
                                dtype=<class 'numpy.int64'>, encoding='utf-8',
                                input='content', lowercase=True, max_df=1.0,
                                max_features=38, min_df=1, ngram_range=(1, 1),
                                preprocessor=None, stop_words=None,
                                strip_accents=None,
                                token_pattern='(?u)\\b\\w\\w+\\b',
                                tokenizer=None, unknown_token=None,
                                vocabulary=None)),
                ('pad',
                 Padder2d(dtype=<class 'int'>, max_len=38, pad_value=38)),
                ('net',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=RequirementRNN(
    (emb): Embedding(39, 128)
    (rec): LSTM(128, 128, num_layers=2, batch_first=True)
    (output): Linear(in_features=128, out_features=2, bias=True)
  ),
))],
         verbose=False)

In [ ]:
#pipe.set_params(net__verbose=0, net__train_split=None)


NameError: ignored

In [ ]:
params = {    
    'module__embedding_dim': [64,128,256],#stats.randint(32, 256 + 1),
    'module__rec_layer_type': ['gru', 'lstm'],
    'module__num_units': [64,128,256],#stats.randint(32, 256 + 1),
    'module__num_layers': [1,2],
    'module__dropout': [0,0.1,0.3],#stats.uniform(0, 0.9),
    'lr': [0.1,0.01,0.001],#[10**(-stats.uniform(1, 5).rvs()) for _ in range(NUM_CV_STEPS)],
    'max_epochs': [3,4,5, 10,30,40],    
}

In [ ]:
search = RandomizedSearchCV(
    modelo_neuronal, params, n_iter=NUM_CV_STEPS, verbose=0, refit=True, scoring='accuracy', cv=10) # se puede agregar otras metricas

In [ ]:
%time search.fit(X_transformado, y)


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7273       0.7557        0.5679  0.0603
      2        0.5657       0.7557        0.5683  0.0548
      3        0.5665       0.7557        0.5566  0.0552
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7016       0.7500        0.5689  0.0512
      2        0.5675       0.7500        0.5680  0.0528
      3        0.5645       0.7500        0.5623  0.0534
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7397       0.7500        0.5768  0.0555
      2        0.5693       0.7500        0.5739  0.0532
      3        0.5660       0.7500        0.5598  0.0534
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7068       0.7

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.RequirementRNN'>,
),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'lr': [0.1, 0.01, 0.001],
                                        'max_epochs': [3, 4, 5, 10, 30, 40],
                                        'module__dropout': [0, 0.1, 0.3],
                                        'module__embedding_dim': [64, 128, 256],
                                        'module__num_layers': [1, 2],
                                        'module__num_units': [64, 128, 256],
                                        'module__rec_layer_type': ['gru',
                                                                   'lstm']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='accuracy', verbose=0)

In [ ]:
search.best_score_, search.best_params_


(0.7800757416368609,
 {'lr': 0.01,
  'max_epochs': 40,
  'module__dropout': 0,
  'module__embedding_dim': 256,
  'module__num_layers': 1,
  'module__num_units': 256,
  'module__rec_layer_type': 'gru'})

In [ ]:
search.best_estimator_

In [ ]:
lista_taggeada = list(df['Tagged_Req'].str.split(','))

In [ ]:
df['Tagged_Req'].head()

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# Display Learning curves to see if overfitting or underfitting data



*   By observing the learning curves, I can tell if the Neural Network overfitted or underfitted the data.
* Overfit : if the training loss curve is significantly lower than the validation loss curve.
* Underfit: if both the training loss curve and the validation loss curve are very high loss.
* Ideal: both the training loss and validation loss curves have a minimal gap between them and converge to a very low loss.




In [ ]:
# get training and validation loss
epochs = [i for i in range(len(search.best_estimator_.history))]
train_loss = search.best_estimator_.history[:,'train_loss']
valid_loss = search.best_estimator_.history[:,'valid_loss']

In [ ]:
plt.plot(epochs,train_loss,'g-');
plt.plot(epochs,valid_loss,'r-');
plt.title('Training Loss Curves');
plt.xlabel('Epochs');
plt.ylabel('Mean Squared Error');
plt.legend(['Train','Validation']);

# See Regression Metrics to evaluate on test dataset

In [ ]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

In [ ]:
# predict on test data
y_pred = search.best_estimator_.predict(X_transformado)

In [ ]:
search.best_estimator_

In [ ]:
MSE(y,y_pred)**(1/2)


In [ ]:
np.array(y)

In [ ]:
y_pred

In [ ]:
sns.kdeplot(y.squeeze(), label='true', shade= True, color = 'blue', linestyle = "dotted")
plt.xlabel('Singularity');

In [ ]:
sns.kdeplot(y_pred.squeeze(), label='estimate', shade=True, color= 'orange')
plt.xlabel('Singularity');

In [ ]:
plt.xlim(-0.15, 0.15)
sns.distplot(y.squeeze()-y_pred.squeeze(),label='error');
plt.xlabel('Singularidad Error') ;

# Codigo para Transformar Requerimientos a Tags

Se importan las librerias

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pandas as pd 
import numpy as np
from nltk.tokenize import word_tokenize

Insertar los requerimientos a continuacion en modo de Lista

In [ ]:
lista_de_req = ['The ATM shall display the Customer Account_Number, Account_ Balance, and so on.','The system shall allow the client to consult third-party accounts added to transfer.']

Se obtienen los tags

In [ ]:
list_word_tokenized = [word_tokenize(str(x)) for x in lista_de_req]
print(list_word_tokenized)
pos_tagged = [nltk.pos_tag(x) for x in list_word_tokenized]
print(pos_tagged)

In [ ]:
lista_taggeada = []
for oracion in pos_tagged:
  oracion_tagged = [] 
  for palabra in oracion:
    oracion_tagged.append(palabra[1])
  lista_taggeada.append(oracion_tagged)

In [ ]:
separator = ','
flat_list = [separator.join(l) for l in lista_taggeada]
print(flat_list)


- CC coordinating conjunction
- CD cardinal digit
- DT determiner
- EX existential there (like: “there is” … think of it like “there exists”)
- FW foreign word
- IN preposition/subordinating conjunction
- JJ adjective ‘big’
- JJR adjective, comparative ‘bigger’
- JJS adjective, superlative ‘biggest’
- LS list marker 1)
- MD modal could, will
- NN noun, singular ‘desk’
- NNS noun plural ‘desks’
- NNP proper noun, singular ‘Harrison’
- NNPS proper noun, plural ‘Americans’
- PDT predeterminer ‘all the kids’
- POS possessive ending parent’s
- PRP personal pronoun I, he, she
- PRP possessive pronoun my, his, hers
- RB adverb very, silently,
- RBR adverb, comparative better
- RBS adverb, superlative best
- RP particle give up
- TO, to go ‘to’ the store.
- UH interjection, errrrrrrrm
- VB verb, base form take
- VBD verb, past tense took
- VBG verb, gerund/present participle taking
- VBN verb, past participle taken
- VBP verb, sing. present, non-3d take
- VBZ verb, 3rd person sing. present takes
- WDT wh-determiner which
- WP wh-pronoun who, what
- WP possessive wh-pronoun whose
- WRB wh-abverb where, when